In [ ]:
pip install gurobipy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 77.7 MB/s eta 0:00:00


In [ ]:
from gurobipy import GRB
import gurobipy as gp
import pandas as pd

Furniture Problem - LP



Objective Function

\begin{equation}
\max revenue = 45x_1 + 80x_2
\end{equation}

Subject to

Mahogany
\begin{equation}
5x_1 + 20x_2 \leq 400
\end{equation}

Labour
\begin{equation}
10x_1 + 15x_2 \leq 450
\end{equation}

Non-negativity
\begin{equation}
x_1, x_2 \geq 0
\end{equation}







In [ ]:
from gurobipy import *

In [ ]:
# Create Furniture new model
f = Model("Furniture")

In [ ]:
# Create decision variables     (Continuous and non-negative)
x1 = f.addVar(name="chairs")
x2 = f.addVar(name="tables")

In [ ]:
# Define objective function (Default value in gurobi is to minimize so we use GRB.MAXIMIZE)
f.setObjective(45*x1 + 80*x2, GRB.MAXIMIZE)

In [ ]:
# Add mahogany constraint
f.addConstr(5*x1 + 20*x2 <= 400, "mahogany")
# Add labour constraint
f.addConstr(10*x1 + 15*x2 <= 450, "labour")

<gurobi.Constr *Awaiting Model Update*>

In [ ]:
f.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xd0437183
Coefficient statistics:
  Matrix range     [5e+00, 2e+01]
  Objective range  [4e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 4e+02]
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.5000000e+31   2.968750e+30   6.500000e+01      0s
       2    2.2000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.03 seconds (0.00 work units)
Optimal objective  2.200000000e+03


In [ ]:
# Display optimal production plan
for v in f.getVars(): #retrieves a list of all variables in the model object f
  print(v.varName, v.x) #The print function displays the decision variable names and solution values

print('Optimal total revenue:', f.objVal) #The print function displays the obj func value of the model object f

chairs 24.0
tables 14.0
Optimal total revenue: 2200.0


Making the problem general to use the model for big data

In [ ]:
\begin{equation}
p \belongs products

\end{equation}

In [ ]:
# resources data
resources, capacity = multidict({
    'mahogany': 400,
    'labour': 450})

In [ ]:
# products data,
products, revenue = multidict({
    'chair': 45,
    'table': 80})

In [ ]:
# bill of materials: resources required by each product
# this dictionary has a 2-tuple as a key, mapping the resource required by a product with its quantity per.
bom = {
    ('mahogany', 'chair'): 5,
    ('mahogany', 'table'): 20,
    ('labour', 'chair'): 10,
    ('labour', 'table'): 15,}

In [ ]:
# Declare and initialize model
f1 = Model('Furniture')

In [ ]:
# Create decision variables for the products to make
# this method adds decision variables to the model object f, and returns a Gurobi tupledict object(make)
# the first argument (products) provides the indices that will be used as keys to access the variables in the returned tupledict.
# the last argument gives the name 'make' to the decision variables.
# The decision variables are of type continuous and non-negative, with no upper bound
make = f1.addVars(products, name="make")

\begin{equation}
\sum_{p \in \text{products}}{\text{bom}_{r,p}}{\text{make}_{p}} \leq {\text{capacity}_{r}} \quad \forall r \in \text{resources}
\end{equation}



In [ ]:
# add constraints (res for resource) to the model object f
res = f1.addConstrs(((sum(bom[r,p]*make[p] for p in products) <= capacity[r])for r in resources), name='R')

\begin{equation}
\text{Max} \sum_{p \in products}{\text{price}_p}{\text{make}_p}
\end{equation}

In [ ]:
# the objective is to maximize total profit
# prod method is the product of object price with the object (make) for each product p in the set of (products)
f1.setObjective(make.prod(revenue), GRB.MAXIMIZE)

In [ ]:
f1.write('furniture.lp')

In [ ]:
# run optimizaiton engine
f1.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xd0437183
Coefficient statistics:
  Matrix range     [5e+00, 2e+01]
  Objective range  [4e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 4e+02]
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

       0    6.5000000e+31   2.968750e+30   6.500000e+01      0s
       2    2.2000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.200000000e+03


In [ ]:
# display optimal values of decision variables
for v in f1.getVars():
  if(abs(v.x) > 1e-6):
    print(v.varName, v.x)

# display optimal total profit value
print('total revenue', f.objVal)

make[chair] 24.0
make[table] 14.0
total revenue 2200.0


#Shadow price: economic interpretation

suppose labour capacity is increased from 450 to 451 hours. what is the increase in the obj function value from such increase?

The simplex method automatically gives the shadow prices.

For each resource constraint in the dictionary (res), check if its associated price is greater than zero. Then print the resource constraint name and the resource constraint shadow price.

In [ ]:
# display shadow prices of resource constraints
for r in res:
  if (abs(res[r].Pi) > 1e-6):
    print(res[r].ConstrName, res[r].Pi)

R[mahogany] 1.0
R[labour] 4.0
